In [1]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'

In [2]:
import os
import json
import joblib

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

from lightfm_wrapper import LightFMWrapper

/Users/peterolsen/.pyenv/versions/csci5123-proj/lib/python3.10/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
df_tracks = pd.read_hdf('df_data/df_tracks.hdf')
df_playlists = pd.read_hdf('df_data/df_playlists.hdf')
df_playlists_info = pd.read_hdf('df_data/df_playlists_info.hdf')
df_playlists_test = pd.read_hdf('df_data/df_playlists_test.hdf')
df_playlists_test_info = pd.read_hdf('df_data/df_playlists_test_info.hdf')

In [4]:
train = pd.read_hdf('df_data/train.hdf')
val = pd.read_hdf('df_data/val1.hdf')
val1_pids = joblib.load('df_data/val1_pids.pkl')

In [5]:
user_seen = train.groupby('pid').tid.apply(set).to_dict()
val_tracks = val.groupby('pid').tid.apply(set).to_dict()

In [6]:
config = {
    'num_playlists': df_playlists_info.pid.max() + 1,
    'num_tracks': df_tracks.tid.max() + 1,
    'model_path': 'models/lightfm_model_text.pkl',
}

In [7]:
zeros_pids = np.array(list(set(val1_pids).difference(train.pid.unique())))

no_zeros_pids = np.array(list(set(val1_pids).difference(zeros_pids))[:1000])
target_pids = np.hstack([zeros_pids, no_zeros_pids])

playlist_name1 = df_playlists_test_info.set_index('pid').name
playlist_name2 = df_playlists_info.set_index('pid').name
playlist_name = pd.concat([playlist_name1, playlist_name2]).sort_index()

In [8]:
playlist_name = playlist_name.reset_index(drop=True)
playlist_name = playlist_name.reindex(np.arange(config['num_playlists'])).fillna('')
# .reindex(np.arange(config['num_playlists']))

vectorizer = CountVectorizer(max_features=20000)
user_features = vectorizer.fit_transform(playlist_name)

user_features = sp.hstack([sp.eye(config['num_playlists']), user_features])

In [9]:
X_train = sp.coo_matrix(
    (np.ones(len(train)), (train.pid, train.tid)),
    shape=(config['num_playlists'], config['num_tracks'])
)

In [10]:
model = LightFMWrapper(
    no_components=200, 
    loss='warp', 
    learning_rate=0.03, 
    max_sampled=400, 
    random_state=1,
    user_alpha=1e-05,
)

In [11]:
best_score = 0

for i in range(10):
    print("iteration:", i)
    model.fit_partial(X_train, epochs=3, num_threads=1, user_features=user_features)

    model.batch_setup(
        item_chunks={0: np.arange(config['num_tracks'])},
        n_process=1, 
        item_features=None,
        user_features=user_features,
    )

    res = model.batch_predict(chunk_id=0, user_ids=target_pids, top_k=600)
    model.batch_cleanup()
    
    score = []
    score2 = []
    
    for pid in zeros_pids:
        tracks_t = val_tracks[pid]
        tracks = [i for i in res[pid][0] if i not in user_seen.get(pid, set())][:len(tracks_t)]
        guess = np.sum([i in tracks_t for i in tracks])
        score.append(guess / len(tracks_t))
    
    for pid in no_zeros_pids:
        tracks_t = val_tracks[pid]
        tracks = [i for i in res[pid][0] if i not in user_seen.get(pid, set())][:len(tracks_t)]
        guess = np.sum([i in tracks_t for i in tracks])
        score2.append(guess / len(tracks_t))
    
    score = np.mean(score)
    score2 = np.mean(score2)
    
    print(score, score2)
    if score > best_score:
        joblib.dump(model, open(config['model_path'], 'wb'))
        best_score = score

iteration: 0
Batch predict: user_ids: 992, item_ids: 113,304
Start recommending: using single process
Recommendations for chunk 0 done in 11.559s. 0.0117 s by user
0.01569837174038791 0.032183292895073115
iteration: 1
Batch predict: user_ids: 992, item_ids: 113,304
Start recommending: using single process
Recommendations for chunk 0 done in 11.475s. 0.0116 s by user
0.011364697191642962 0.037951468752568294
iteration: 2
Batch predict: user_ids: 992, item_ids: 113,304
Start recommending: using single process
Recommendations for chunk 0 done in 11.671s. 0.0118 s by user
0.010265488154177889 0.04037701593614937
iteration: 3
Batch predict: user_ids: 992, item_ids: 113,304
Start recommending: using single process
Recommendations for chunk 0 done in 12.000s. 0.0121 s by user
0.009857091197963587 0.0405445940764208
iteration: 4
Batch predict: user_ids: 992, item_ids: 113,304
Start recommending: using single process
Recommendations for chunk 0 done in 11.474s. 0.0116 s by user
0.00992515916060

In [12]:
joblib.dump(user_features, open('models/user_features.pkl', 'wb'))

In [41]:
model = joblib.load(open(config['model_path'], 'rb'))